In [2]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine, inspect, text

### Database connections

In [3]:
# Cargar configuración desde el archivo config.yml
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones a las bases de datos
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear SQLAlchemy engine
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)


# Extract

### SQL Query

In [4]:
# Consulta SQL para obtener los estados
query = """
SELECT 
    id as estado_id,
    nombre as nombre_estado,
    descripcion
FROM mensajeria_estado
"""

In [5]:
# Leer datos de la fuente
dim_estado = pd.read_sql(query, fuente_conn)

# Transformation

In [6]:
# Agregar fecha de carga
dim_estado['saved'] = date.today()

### Verificaciones

In [7]:
# Verificar los datos
print("\nInformación del DataFrame:")
print(dim_estado.info())

print("\nMuestra de los datos:")
print(dim_estado.head())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   estado_id      6 non-null      int64 
 1   nombre_estado  6 non-null      object
 2   descripcion    6 non-null      object
 3   saved          6 non-null      object
dtypes: int64(1), object(3)
memory usage: 324.0+ bytes
None

Muestra de los datos:
   estado_id           nombre_estado             descripcion       saved
0          4  Recogido por mensajero  Recogido por mensajero  2024-11-10
1          5    Entregado en destino    Entregado en destino  2024-11-10
2          3             Con novedad           Tiene novedad  2024-11-10
3          6      Terminado completo      Terminado completo  2024-11-10
4          1                Iniciado   Creado por el usuario  2024-11-10


In [8]:
# Verificaciones adicionales
print("\nNúmero total de estados:", len(dim_estado))
print("\nEstados únicos:")
print(dim_estado[['estado_id', 'nombre_estado']].to_string())


Número total de estados: 6

Estados únicos:
   estado_id           nombre_estado
0          4  Recogido por mensajero
1          5    Entregado en destino
2          3             Con novedad
3          6      Terminado completo
4          1                Iniciado
5          2  Con mensajero Asignado


# Load

In [9]:
# Cargar en la bodega
dim_estado.to_sql('dim_estado', bodega_conn, if_exists='replace', index_label='key_dim_estado')

6